In this notebook we will show a simple example of language model prompting using API calls.

In [1]:
# Import required libraries
import anthropic
import os

# Set up the API client
# Make sure to set your API key as an environment variable: ANTHROPIC_API_KEY
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC")
)


### Simple walk-through

In [2]:
model = "claude-3-5-haiku-latest"
max_tokens = 1000 
prompt = "What is the capital of France?"

message = client.messages.create(
    model=model,
    max_tokens=max_tokens,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message

Message(id='msg_017zRJf5pnmqdKLqLuryrFbL', content=[TextBlock(citations=None, text='The capital of France is Paris.', type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=14, output_tokens=10, server_tool_use=None, service_tier='standard'))

In [3]:
message.content[0].text

'The capital of France is Paris.'

Have the API return the result as a JSON object, containing only the name of the capital.  First let's create a function to submit prompts to claude, in order to simplify our interactions:

In [3]:
from bettercode.llm_utils import send_prompt_to_claude

json_prompt = """
What is the capital of France? 

Please return your response as a JSON object with the following structure:
{
    "capital": "city_name",
    "country": "country_name"
}
"""

result = send_prompt_to_claude(json_prompt, client)
result

'{\n    "capital": "Paris",\n    "country": "France"\n}'

In [4]:
import json

result_dict = json.loads(result)
result_dict

{'capital': 'Paris', 'country': 'France'}

In [5]:
# Run on multiple countries

countries = ["France", "Germany", "Spain", "Italy", "Portugal", "Netherlands", "Belgium", "Sweden", "Norway", "Finland"]

ntokens_loop = 0
for country in countries:
    json_prompt = f"""
    What is the capital of {country}? 

    Please return your response as a JSON object with the following structure:
    {{
        "capital": "city_name",
        "country": "country_name"
    }}
    """


    result, ntokens_prompt = send_prompt_to_claude(json_prompt, client, return_tokens=True)
    ntokens_loop += ntokens_prompt
    result_dict = json.loads(result)
    print(result_dict)


{'capital': 'Paris', 'country': 'France'}
{'capital': 'Berlin', 'country': 'Germany'}
{'capital': 'Madrid', 'country': 'Spain'}
{'capital': 'Rome', 'country': 'Italy'}
{'capital': 'Lisbon', 'country': 'Portugal'}
{'capital': 'Amsterdam', 'country': 'Netherlands'}
{'capital': 'Brussels', 'country': 'Belgium'}
{'capital': 'Stockholm', 'country': 'Sweden'}
{'capital': 'Oslo', 'country': 'Norway'}
{'capital': 'Helsinki', 'country': 'Finland'}


In [12]:
json_prompt_all = f"""
Here is a list of countries:
{', '.join(countries)}

For each country, please provide the capital city in a 
JSON object with the country name as the key and the 
capital city as the value.  

IMPORTANT: Return only the JSON object without any additional text.
"""
result_all, ntokens_prompt = send_prompt_to_claude(json_prompt_all, client, return_tokens=True)


In [13]:
result_all

'{\n    "France": "Paris",\n    "Germany": "Berlin",\n    "Spain": "Madrid",\n    "Italy": "Rome",\n    "Portugal": "Lisbon",\n    "Netherlands": "Amsterdam",\n    "Belgium": "Brussels",\n    "Sweden": "Stockholm",\n    "Norway": "Oslo",\n    "Finland": "Helsinki"\n}'

In [16]:
print(ntokens_prompt)
print(ntokens_loop)

172
832
